In [18]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import metrics
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [36]:
class BinaryClassficationNeuralNet:
    def  __init__(self,data,hl):
        self.x_train,self.x_test, self.y_train, self.y_test = data
        # (self.x_test, self.y_test) = data[1] 
        print(self.x_train.shape)
        print(self.y_train.shape)
        print(self.x_test.shape)
        print(self.y_test.shape)
        m,n = self.x_train.shape
        self.cols = n
        self.rows = m
        self.hiddenLayers = hl
        self.create()

    def create(self):
        if(len(self.hiddenLayers)):
            inp_layer = [keras.layers.Dense(self.hiddenLayers[0],input_dim = self.cols)]
            self.hiddenLayers.remove(self.hiddenLayers[0])
            hidden_layers = [keras.layers.Dense(units=l, activation='relu') for l in self.hiddenLayers]

        else:
            inp_layer = [keras.layers.Dense(self.cols,input_dim = self.cols)]
        op_layer = [keras.layers.Dense(units=1, activation='sigmoid')]
        
        self.model = keras.Sequential(inp_layer+hidden_layers+op_layer)
        self.model.compile(loss='binary_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'])

    def fit(self,EPOCHS,BATCH_SIZE):
         self.history = self.model.fit(
                            x=self.x_train,
                            y =self.y_train, 
                            batch_size=BATCH_SIZE,
                            epochs=EPOCHS, 
                            validation_data=(self.x_test, self.y_test), verbose=0
                        )

    def predict(self,X):
        return model.predict(X)

    def evaluate(self):
        e = self.model.evaluate(self.x_test,self.y_test)
        print(e)
        return e

    def get_weights(self):
        return self.model.get_weights()

    def print_summary(self):
        print(self.model.summary())

In [37]:
def dataPreprocess(df,features):
    #replacing zeros with mean of that features
    zero_fields = []
    
    def check_zero_entries(data, fields):
        for field in fields:
            print('field %s: num 0-entries: %d' % (field, len(data.loc[ data[field] == 0, field ])))
            if len(data.loc[ data[field] == 0, field ]) !=0 :
                zero_fields.append(field)
        return zero_fields

    zero_fields = check_zero_entries(df, features)
    print(zero_fields)

    def impute_zero_field(data, field):
        nonzero_vals = data.loc[data[field] != 0, field]
        avg = np.sum(nonzero_vals) / len(nonzero_vals)
        k = len(data.loc[ data[field] == 0, field])   # num of 0-entries
        data.loc[ data[field] == 0, field ] = avg
        print('Field: %s; fixed %d entries with value: %.3f' % (field, k, avg))

    for field in zero_fields:
        impute_zero_field(df, field)

    # check_zero_entries(df, zero_fields)

In [38]:
def getDS(filename):
    df = pd.read_csv(filename)  
    features = list(df.columns.values)
    X = df[features[0:-1]]
    y = df[features[-1]]
    dataPreprocess(df,features[0:-1])
    return train_test_split(X, y,test_size=0.2)



In [39]:
nn = BinaryClassficationNeuralNet(getDS('./diabetes.csv'),[16,12,8,4])

field Pregnancies: num 0-entries: 111
field Glucose: num 0-entries: 5
field BloodPressure: num 0-entries: 35
field SkinThickness: num 0-entries: 227
field Insulin: num 0-entries: 374
field BMI: num 0-entries: 11
field DiabetesPedigreeFunction: num 0-entries: 0
field Age: num 0-entries: 0
['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
Field: Pregnancies; fixed 111 entries with value: 4.495
Field: Glucose; fixed 5 entries with value: 121.687
Field: BloodPressure; fixed 35 entries with value: 72.405
Field: SkinThickness; fixed 227 entries with value: 29.153
Field: Insulin; fixed 374 entries with value: 155.548
Field: BMI; fixed 11 entries with value: 32.457
(614, 8)
(614,)
(154, 8)
(154,)


In [34]:
nn.fit(500,16)

Train on 614 samples, validate on 154 samples
Epoch 1/500
614/614 [==============================] - 3s 4ms/sample - loss: 2.3039 - accuracy: 0.6026 - val_loss: 1.1327 - val_accuracy: 0.6104
Epoch 2/500
614/614 [==============================] - 0s 579us/sample - loss: 1.1609 - accuracy: 0.6352 - val_loss: 0.8074 - val_accuracy: 0.6364
Epoch 3/500
614/614 [==============================] - 0s 558us/sample - loss: 0.9639 - accuracy: 0.6417 - val_loss: 0.8795 - val_accuracy: 0.6169
Epoch 4/500
614/614 [==============================] - 0s 615us/sample - loss: 0.9751 - accuracy: 0.6336 - val_loss: 1.1017 - val_accuracy: 0.6299
Epoch 5/500
614/614 [==============================] - 0s 552us/sample - loss: 0.7969 - accuracy: 0.6368 - val_loss: 0.6441 - val_accuracy: 0.6883
Epoch 6/500
614/614 [==============================] - 0s 672us/sample - loss: 0.6804 - accuracy: 0.6319 - val_loss: 0.6073 - val_accuracy: 0.6818
Epoch 7/500
614/614 [==============================] - 0s 491us/sample - l

In [35]:
nn.evaluate()

154/154 [==============================] - 0s 215us/sample - loss: 0.5504 - accuracy: 0.7662
[0.5503779417508609, 0.76623374]


[0.5503779417508609, 0.76623374]